In [12]:
!python -m venv env

!source env/bin/activate

!pip install openai
!pip install python-docx

!apt-get install -y libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0
!pip install sounddevice
!pip install soundfile

!brew install ffmpeg
!pip install pydub

!pip install numpy

You should consider upgrading via the '/Users/jackson/Desktop/Spring 2024/Advanced ML/ElmTA_project/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jackson/Desktop/Spring 2024/Advanced ML/ElmTA_project/env/bin/python -m pip install --upgrade pip' command.
zsh:1: command not found: apt-get
You should consider upgrading via the '/Users/jackson/Desktop/Spring 2024/Advanced ML/ElmTA_project/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jackson/Desktop/Spring 2024/Advanced ML/ElmTA_project/env/bin/python -m pip install --upgrade pip' command.
To reinstall 7.0_1, run:
  brew reinstall ffmpeg
You should consider upgrading via the '/Users/jackson/Desktop/Spring 2024/Advanced ML/ElmTA_project/env/bin/python -m pip install --upgrade pip' command.
  Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl (19.8 MB)
You should consider upgrading via the '/Users/jackson/Desktop/Spring 2024/Ad

In [2]:
from openai import OpenAI

client = OpenAI(  
    api_key="sk-proj-aqSU5HlKWDHScsLaSj3QT3BlbkFJwawlJWzFM0VC0jRHzbMN" ##Personal api key
)
from docx import Document

async def transcribe_audio():
    # Transcribe the audio using the OpenAI API
    with open('recorded_audio.wav', 'rb') as f:
        transcription = client.audio.transcriptions.create(model="whisper-1", file=f, response_format='text')

    return transcription

In [3]:
def generate_response(transcription):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a highly skilled AI trained in language comprehension and summarization. I would like you to read the following text and respond in a manner of a elmentray school teacher. Aim to put things in simple terms, providing a coherent and easy to understand response such that a child could understand. Please ensure that your reponse would follow a school's policy. If the student's question is something they can solve or is part of homework, please guide them with steps to take to the solution without directly providing it."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response.choices[0].message.content

In [4]:
def analyze_question(transcription):
    response= generate_response(transcription)
    # key_points = key_points_extraction(transcription)
    # action_items = action_item_extraction(transcription)
    # sentiment = sentiment_analysis(transcription)
    return {
        'response': response,
        # 'key_points': key_points,
        # 'action_items': action_items,
        # 'sentiment': sentiment
    }

In [13]:
import asyncio
import sounddevice as sd
import numpy as np
import wave

async def record_audio(duration):
    # Define recording parameters
    fs = 44100  # Sample rate
    seconds = duration  # Duration of recording
    
    # Record audio
    print("Recording audio...")
    audio_data = sd.rec(int(seconds * fs), samplerate=fs, channels=1, dtype="int16")
    sd.wait()  # Wait until recording is finished
    
    # Save the recorded audio to a WAV file
    print("Audio recorded successfully.")
    with wave.open('recorded_audio.wav', 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(fs)
        wf.writeframes(audio_data.tobytes())
    


In [14]:
from IPython.display import Audio, display
import tempfile


def text_to_speech(text):
    # Call OpenAI's TTS API
    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=text
    )

   # Read the audio data from the response
    audio_data = response.content

    # Save the audio data to a temporary WAV file
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_wav_file:
        temp_wav_file.write(audio_data)
        temp_wav_file_name = temp_wav_file.name

    # Display the speech as an audio widget
    display(Audio(filename=temp_wav_file_name, autoplay=True))



In [16]:
async def main():
    # Record audio for 8 seconds
    await record_audio(duration=8)

    # Wait for the audio recording to finish
    await asyncio.sleep(2)
    
    # Transcribe the recorded audio
    transcription = await transcribe_audio()

    # Print the transcribed audio
    print("Transcription:", transcription)

    # Analyze the transcription
    ElmAnswer = analyze_question(transcription)

    # Print the answer
    print("Answer:", ElmAnswer['response'])

    # Convert the response to speech and play it
    text_to_speech(ElmAnswer['response'])

# Run the main function
await main()


Recording audio...
Audio recorded successfully.
Transcription: How do I design a nuclear fusion reactor? Specifically, how do I design a tokamak?

Answer: Wow, that's a very advanced question! Nuclear fusion and tokamaks are topics that even some adults find challenging. But let's try to simplify it a bit.

Nuclear fusion is like a special recipe where we take two lighter atoms, like hydrogen, and combine them to make a heavier atom, like helium. This process releases a lot of energy, which is what powers the sun and stars. Scientists are trying to use this process to create a new source of energy on Earth.

A tokamak is a machine that scientists use to create and control a nuclear fusion reaction. It's shaped like a donut and uses strong magnetic fields to keep the hot plasma (a state of matter like gas, but even hotter) inside without touching the walls.

Designing a tokamak is a very complex task that requires a deep understanding of physics, engineering, and mathematics. Here are s